In [2]:
def log_progress(sequence, every=None, size=None):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = size / 200     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    label.value = u'{index} / {size}'.format(
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = str(index or '?')

In [3]:
def jobs_manager():
    from IPython.lib.backgroundjobs import BackgroundJobManager
    from IPython.core.magic import register_line_magic
    from IPython import get_ipython
    
    jobs = BackgroundJobManager()

    @register_line_magic
    def job(line):
        ip = get_ipython()
        jobs.new(line, ip.user_global_ns)

    return jobs

In [4]:
def kill_thread(thread):
    import ctypes
    
    id = thread.ident
    code = ctypes.pythonapi.PyThreadState_SetAsyncExc(
        ctypes.c_long(id),
        ctypes.py_object(SystemError)
    )
    if code == 0:
        raise ValueError('invalid thread id')
    elif code != 1:
        ctypes.pythonapi.PyThreadState_SetAsyncExc(
            ctypes.c_long(id),
            ctypes.c_long(0)
        )
        raise SystemError('PyThreadState_SetAsyncExc failed')

In [5]:
def get_chunks(sequence, count):
    count = min(count, len(sequence))
    chunks = [[] for _ in range(count)]
    for index, item in enumerate(sequence):
        chunks[index % count].append(item) 
    return chunks

In [6]:
jobs = jobs_manager()

In [21]:
from time import sleep
from random import random

def fetch_url(url):
    sleep(random())
    return url

urls = range(100)
urls2 = range(100)

In [13]:
%job [fetch_url(_) for _ in log_progress(urls, every=1)]

Starting job # 2 in a separate thread.


In [15]:
kill_thread(jobs.running[0])

In [18]:
for chunk in get_chunks(urls, 3):
    %job [fetch_url(_) for _ in log_progress(chunk, every=1)]

Starting job # 8 in a separate thread.
Starting job # 9 in a separate thread.
Starting job # 10 in a separate thread.


In [19]:
for job in jobs.running:
    kill_thread(job)

In [14]:
jobs = jobs_manager()

In [15]:
%job [fetch_url(_) for _ in log_progress(urls, every=1)]

Starting job # 0 in a separate thread.


In [28]:
%job [fetch_url(_) for _ in log_progress(urls, every=1)]

Starting job # 7 in a separate thread.


In [29]:
kill_thread(jobs.all[7])

In [30]:
jobs.traceback(7)

---------------------------------------------------------------------------
SystemError                               Traceback (most recent call last)
/Users/alexkuk/envs/mypy/lib/python2.7/site-packages/IPython/lib/backgroundjobs.pyc in call(self)
    463 
    464     def call(self):
--> 465         return eval(self.code,self.glob,self.loc)
    466 
    467 

<BackgroundJob compilation> in <module>()

<ipython-input-2-dda97720bb84> in log_progress(sequence, every, size)
     34                     label.value = '{index} / ?'.format(index=index)
     35                 else:
---> 36                     progress.value = index
     37                     label.value = u'{index} / {size}'.format(
     38                         index=index,

/Users/alexkuk/envs/mypy/lib/python2.7/site-packages/traitlets/traitlets.pyc in __set__(self, obj, value)
    556             raise TraitError('The "%s" trait is read-only.' % self.name)
    557         else:
--> 558             self.set(obj, value)


In [34]:
for chunk in get_chunks(urls, 3):
    %job [fetch_url(_) for _ in log_progress(chunk, every=1)]

Starting job # 17 in a separate thread.
Starting job # 18 in a separate thread.
Starting job # 19 in a separate thread.


In [35]:
for thread in jobs.running:
    kill_thread(thread)

In [36]:
eval('a = 1')

SyntaxError: invalid syntax (<string>, line 1)